# DAM

In [1]:
from fastai.vision.all import *
import pandas as pd
import cam
import util

In [2]:
base_model_name = 'densenet121-chexpert-final'

dls, labels = util.chexpert_data_loader(bs=16)

In [3]:
# Load the base model
model_base = cnn_learner(dls, densenet121, n_out=len(labels), path=Path('../saves/'), y_range=(0,1))
model_base.load(base_model_name)

# Set all parameters to require grad
for param in model_base.parameters():
    param.requires_grad = True
    
# Initialize DAM learner
dam_learner = util.DAM(model_base, dls, folder=Path('../saves/densenet121-dam'), lr=0.04)

In [4]:
print(f'Initial AUC score: {dam_learner.eval()}')

Initial AUC score: 0.8167849105903288


In [ ]:
dam_learner.train(max_epoch=3, lr_div=1.25)

epoch: 0, train_loss: -0.016421, train_auc:0.762398, test_auc:0.813186, lr:0.040000
Update Regularizer @ T=11182!
epoch: 1, train_loss: -0.004450, train_auc:0.749058, test_auc:0.817175, lr:0.032000
Update Regularizer @ T=11182!


Get the categorywise AUC scores of the best model.

In [4]:
dam_learner.load('m-epoch 2-2021_May_10-02_19_17.pth.tar')

In [5]:
categorywise_auc_scores = dam_learner.eval(average=None)
print(categorywise_auc_scores)

[0.87594719 0.65596326 0.84921414 0.73102591 0.76650035 0.8407368
 0.72227811 0.74685532 0.6878983  0.86726454 0.87525643 0.79727297
 0.73168564 0.88334658]


In [6]:
idx_5_labels = (2, 5, 6, 8, 10)
avg_auc = 0
for i in idx_5_labels:
    avg_auc += categorywise_auc_scores[i]
avg_auc /= len(idx_5_labels)
print(avg_auc)

0.795076755911267
